<a href="https://colab.research.google.com/github/minoxd/fiber_optic_box_inspection/blob/main/hehe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Environmental Setup

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
!pip install ultralytics
from ultralytics import YOLO

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 978.8/978.8 kB 47.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 123.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 98.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 44.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 106.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninst

In [ ]:
import os
import shutil
import zipfile
import tarfile
import json
from sklearn.model_selection import train_test_split
import yaml
import torch
from multiprocessing import Pool
import pandas as pd

In [ ]:
BASE_DIR = '/content/drive/MyDrive/fiber_optic_box_inspection'
if not os.path.exists(BASE_DIR):
    raise FileNotFoundError(f"Base directory {BASE_DIR} not found. Check Google Drive mount.")

paths = {
    'Data_INF':                   os.path.join(BASE_DIR, 'Data_INF'),
    'dataset_dir':                os.path.join(BASE_DIR, 'dataset'),
    'yolo_output_dir':            os.path.join(BASE_DIR, 'runs_yolo'),
}
paths['raw_classes'] =            os.path.join(paths['Data_INF'], 'inf_raw_classes.json')
paths['matching_name_id'] =       os.path.join(paths['Data_INF'], 'matching_name_id_v10.0.xlsx')
paths['data_label_compressed'] =  os.path.join(paths['Data_INF'], 'data_label_ALL.tar.gz')

paths['data_label_dir'] =         os.path.join(paths['dataset_dir'], 'data_label_v2_ALL')
paths['backup_label_dir'] =       os.path.join(paths['data_label_dir'], 'backup_label')
if not os.path.exists(paths['backup_label_dir']):
    os.makedirs(paths['backup_label_dir'])

paths['data'] =                   os.path.join(paths['dataset_dir'], 'data.yaml')
paths['all_paths'] =              os.path.join(paths['dataset_dir'], 'all.txt')
paths['train_paths'] =            os.path.join(paths['dataset_dir'], 'train.txt')
paths['val_paths'] =              os.path.join(paths['dataset_dir'], 'val.txt')
paths['test_paths'] =             os.path.join(paths['dataset_dir'], 'test.txt')

In [ ]:
with open(paths['raw_classes'], 'r') as f:
    class_dict = json.load(f)
print("Class dictionary from JSON:", class_dict)

class_ids_from_json = sorted([int(k) for k in class_dict.keys()])
class_names = [class_dict[str(id)] if id in class_ids_from_json else '' for id in range(class_ids_from_json[-1] + 1)]
print("Class names in order:", class_names)

Class dictionary from JSON: {'0': '1.0', '1': '1.1', '2': '1.2', '4': '1.7', '7': '2.0', '8': '2.1', '9': '2.3', '10': '3.0', '11': '3.1', '12': '4.0', '13': '4.1', '14': '5.0', '15': '5.1', '16': '6.0', '17': '6.1', '18': '7.0', '19': '7.1', '20': '7.2', '21': '7.3', '23': '7.7', '24': '8.0', '25': '8.3', '27': '8.5', '28': '9.0', '30': '9.4', '32': '10.0', '33': '10.1', '34': '11.0', '35': '11.1', '36': '13.0', '37': '13.1', '38': '13.2', '41': '14.0', '42': '14.1', '43': '14.2', '44': '14.3', '45': '15.0', '46': '15.1', '47': '15.4', '50': '17.0', '51': '17.1', '52': '18.0', '53': '18.1', '54': '19.0', '55': '19.1', '56': '19.2', '57': '19.3', '58': '19.4', '59': '19.5', '60': '20.0', '61': '20.1', '62': '20.2', '63': '20.3', '67': '23.0', '68': '23.1', '69': '23.2', '70': '24.0', '71': '24.1', '72': '24.3', '73': '26.0', '74': '26.1', '75': '27.0', '76': '27.1', '77': '28.0', '78': '28.3', '80': '29.0', '81': '29.1', '82': '29.2', '83': '29.4', '84': '30.0', '85': '30.1', '86': '32

In [ ]:
matching_name_id_df

,id,title,Unnamed: 2
0,0.1,"Ảnh mờ, sai góc chụp",Bổ sung mới
1,0.2,"Ảnh giả, ảnh chụp qua màn hình",Bổ sung mới
2,1.0,Tên tập điểm: sơn đỏ/đen(Đạt),NaN
3,1.1,Tên bị mờ,Không thay đổi
4,1.2,"Giấy dán quảng cáo che, chưa vệ sinh sạch","Cập nhật tên, không thay đổi gì thêm"
...,...,...,...
181,71.1,Modem cũ/ bẩn,Bổ sung mới
182,72.1,Có/Không có dây LAN,Bổ sung mới
183,72.2,Có/không có dây LAN đúc sẵn,Bổ sung mới
184,73.1,Giá trị đo công suất tập điểm,Bổ sung mới


In [ ]:
matching_name_id_df = pd.read_excel(paths['matching_name_id'], sheet_name=1)
matching_name_id = dict(zip(matching_name_id_df['id'], matching_name_id_df['title']))
print(matching_name_id[0.1])

Ảnh mờ, sai góc chụp


In [ ]:
id_to_names = matching_name_id_df.groupby(matching_name_id_df['id'].astype(int))['title'].apply(list).to_dict()

def find_names_by_id(integer_part):
    return id_to_names.get(int(integer_part), ["ID not found"])

integer_part = 0
names = find_names_by_id(integer_part)
print(f"Names for IDs with integer part {integer_part}: {names}")

Names for IDs with integer part 0: ['Ảnh mờ, sai góc chụp', 'Ảnh giả, ảnh chụp qua màn hình']


In [ ]:
id_to_names

{0: ['Ảnh mờ, sai góc chụp', 'Ảnh giả, ảnh chụp qua màn hình'],
 1: ['Tên tập điểm: sơn đỏ/đen(Đạt)',
  'Tên bị mờ',
  'Giấy dán quảng cáo che, chưa vệ sinh sạch',
  'Không sơn tên',
  'Sơn sai quy cách'],
 2: ['Nắp tập điểm(Đạt)', 'Mất nắp tập điểm', 'Vỡ nắp tập điểm, biến dạng'],
 3: ['Ốc khóa bán nguyệt(Đạt)', 'Mất ốc khóa bán nguyệt'],
 4: ['Ốc vít khóa hộp(Đạt)', 'Mất ốc vít khóa hộp'],
 5: ['Tai nhựa đóng/mở hộp (2 tai)(Đạt)', 'Vỡ tai nhựa đóng/mở hộp'],
 6: ['Bản lề tập điểm(Đạt)', 'Vỡ bản lề tập điểm'],
 7: ['Ống nhựa luồn cáp(Đạt)',
  'Bắt cao/thấp so với đáy tập điểm',
  'Không có ống nhựa luồn cáp',
  'Ống nhựa luồn cáp đặt không đúng bên trái hộp',
  'Vỡ ống nhựa luồn cáp'],
 8: ['Đai inox(Đạt)', 'Đai inox đánh không chặt', 'Đai inox bị mất'],
 9: ['Khóa đai inox(Đạt)',
  'Khóa đai lắp không đúng vị trí theo quy định (lắp trên ống nhựa)'],
 10: ['Khung đáy tập điểm(Đạt)', 'Khung thân tập điểm vỡ'],
 11: ['Gioăng cao su chống nước(Đạt)',
  'Gioăng cao su chống nước bị mất ho

# Data Preparation (Run once per lifetime)

## Extract compressed dataset

In [ ]:
with tarfile.open(paths['data_label_compressed'], 'r:gz') as tar_ref:
    tar_ref.extractall(paths['dataset_dir'])
    print(f"Extracted data_label_ALL.tar.gz to {paths['dataset_dir']}")

Extracted data_label_ALL.tar.gz to /content/drive/MyDrive/fiber_optic_box_inspection/dataset


## Split dataset & save dataset division

In [ ]:
train_ratio = 0.7
val_ratio = 0.15
test_ratio = 0.15

all_images = [os.path.join(paths['data_label_dir'], f) for f in os.listdir(paths['data_label_dir']) if f.endswith('.jpg')]

train_images, temp_images = train_test_split(all_images, train_size=train_ratio, random_state=42)
val_images, test_images = train_test_split(temp_images, train_size=(val_ratio / (val_ratio + test_ratio)), random_state=42)

In [ ]:
with open(paths['all_paths'], "w") as f:
    for img in all_images:
        f.write(f"{img}\n")

with open(paths['train_paths'], "w") as f:
    for img in train_images:
        f.write(f"{img}\n")

with open(paths['val_paths'], "w") as f:
    for img in val_images:
        f.write(f"{img}\n")

with open(paths['test_paths'], "w") as f:
    for img in test_images:
        f.write(f"{img}\n")

In [ ]:
# todo this needed where?
def read_image_paths(file_path):
    if os.path.exists(file_path):
        with open(file_path, "r") as f:
            return [line.strip() for line in f if line.strip()]
    return []

all_images = read_image_paths(paths['all_paths'])
train_images = read_image_paths(paths['train_paths'])
val_images = read_image_paths(paths['val_paths'])
test_images = read_image_paths(paths['test_paths'])

## Validate class ids

In [ ]:
def get_class_ids_from_label_file(label_file):
    class_ids = set()
    try:
        with open(label_file, 'r') as f:
            for line in f:
                if line.strip():
                    class_id = line.strip().split()[0]
                    class_ids.add(class_id)
    except Exception as e:
        print(f"Error processing {label_file}: {e}")
    return class_ids

def get_class_ids_from_label_files(label_dir):
    label_files = [os.path.join(label_dir, f) for f in os.listdir(label_dir) if f.endswith('.txt')]
    with Pool() as pool:
        results = pool.map(get_class_ids_from_label_file, label_files)
    class_ids = set().union(*results)
    return sorted([int(id) for id in class_ids])

In [ ]:
class_ids_from_label_files = get_class_ids_from_label_files(paths['data_label_dir'])

In [ ]:
class_ids_from_label_files_not_in_json = [id for id in class_ids_from_label_files if id not in class_ids_from_json]

In [ ]:
print(f'Class IDs from json:                       \n\t {class_ids_from_json}')
print(f'Class IDs from label files:                \n\t {class_ids_from_label_files}')
print(f'Class Ids from label files but not in json:\n\t {class_ids_from_label_files_not_in_json}')

Class IDs from json:                       
	 [0, 1, 2, 4, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 23, 24, 25, 27, 28, 30, 32, 33, 34, 35, 36, 37, 38, 41, 42, 43, 44, 45, 46, 47, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 80, 81, 82, 83, 84, 85, 86, 87, 88, 93, 94, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172]
Class IDs from label files:                
	 [0, 1, 2, 3, 4, 7, 9, 10, 11, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 32, 33, 34, 35, 36, 37, 38, 39, 41, 42, 43, 44, 54, 55, 56, 57, 58, 60, 61, 62, 63, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 80, 81, 82, 

In [ ]:
from_json = [0, 1, 2, 4, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 23, 24, 25, 27, 28, 30, 32, 33, 34, 35, 36, 37, 38, 41, 42, 43, 44, 45, 46, 47, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 80, 81, 82, 83, 84, 85, 86, 87, 88, 93, 94, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172]
omitted = [id for id in range(173) if id not in from_json]
print(omitted)
print(len(from_json))
print(len(omitted))

[3, 5, 6, 22, 26, 29, 31, 39, 40, 48, 49, 64, 65, 66, 79, 89, 90, 91, 92, 95, 96, 121]
151
22


In [ ]:
def process_label_file(args):
    label_file, label_dir, backup_dir, class_ids_to_remove = args
    filename = os.path.basename(label_file)
    try:
        with open(label_file, 'r') as f:
            lines = f.readlines()

        lines_kept = [
            line for line in lines
            if line.strip() and line.strip().split()[0] not in class_ids_to_remove
        ]

        lines_removed = len(lines) - len(lines_kept)

        if lines_removed <= 0:
            return

        backup_file = os.path.join(backup_dir, filename)
        shutil.copy(label_file, backup_file)
        with open(label_file, 'w') as f:
            f.writelines(lines_kept)

        return f"Processed {filename}: {lines_removed} lines removed, original backed up"
    except Exception as e:
        return f"Error processing {filename}: {e}"

def process_label_files(label_dir, backup_dir, class_ids_to_remove):
    class_ids_to_remove = [str(id) for id in class_ids_to_remove]

    all_labels = [
        os.path.join(label_dir, f) for f in os.listdir(label_dir)
        if f.endswith('.txt')
    ]

    args = [(f, label_dir, backup_dir, class_ids_to_remove) for f in all_labels]

    with Pool() as pool:
        results = pool.map(process_label_file, args)

    for result in results:
        if result:
            print(result)

    return len(all_labels)

In [ ]:
num_files_processed = process_label_files(
    label_dir=paths['data_label_dir'],
    backup_dir=paths['backup_label_dir'],
    class_ids_to_remove=class_ids_from_label_files_not_in_json
)
print(f"Total files processed: {num_files_processed}")

Processed 2022_11_16_15_16_02-BPCP002389HO.txt: 1 lines removed, original backed up
Processed 2022_11_16_11_57_38-HGGP014205HO.txt: 1 lines removed, original backed up
Processed 2022_11_19_11_19_42-BPCP008009HO.txt: 1 lines removed, original backed up
Processed 2022_11_18_11_36_22-BPCP008021HO.txt: 3 lines removed, original backed up
Processed 2022_11_19_08_53_38-DNIP106460HO.txt: 1 lines removed, original backed up
Processed 2022_11_09_11_01_01-TNHP036130HO.txt: 3 lines removed, original backed up
Processed 2022_11_16_11_13_19-HGGP004212HO.txt: 1 lines removed, original backed up
Processed 2022_11_08_14_33_23-TNHP007376HO.txt: 2 lines removed, original backed up
Processed 2022_11_09_07_55_41-TNHP007376HO.txt: 2 lines removed, original backed up
Processed 2022_11_03_08_42_50-DNIP106463HO.txt: 1 lines removed, original backed up
Processed 2022_11_08_15_36_36-BRUP040352HO.txt: 2 lines removed, original backed up
Processed 2022_11_08_14_19_52-TNHP007376HO.txt: 1 lines removed, original ba

## Write dataset configuration

In [ ]:
data = {
    "train": paths['train_paths'],
    "val": paths['val_paths'],
    "test": paths['test_paths'],
    "nc": len(class_names),
    "names": class_names
}

with open(paths['data'], "w") as file:
    yaml.dump(data, file, default_flow_style=False)

print("data.yaml has been generated!")

data.yaml has been generated!


# Model Training

## GPU check

In [ ]:
!nvidia-smi

Fri Apr 18 08:51:43 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA L4                      Off |   00000000:00:03.0 Off |                    0 |
| N/A   36C    P8             11W /   72W |       0MiB /  23034MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"GPU is available! Using device: {device}")
    print(f"GPU Name: {torch.cuda.get_device_name(0)}")
    print(f"Number of GPUs: {torch.cuda.device_count()}")
    print(f"Total Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
else:
    device = torch.device("cpu")
    print("No GPU available, using CPU.")


GPU is available! Using device: cuda
GPU Name: NVIDIA L4
Number of GPUs: 1
Total Memory: 23.80 GB


## Train

In [ ]:
model = YOLO('yolov8m.pt')
model.train(
    data=paths['data'],
    epochs=100,
    patience=10,
    batch=25,
    imgsz=896,
    save_period=5,
    cache=True,
    device=0,
    workers=16,
    project=paths['yolo_output_dir'],
    name='v8m_batch25_imgsz896_',)

Ultralytics 8.3.110 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (NVIDIA L4, 22693MiB)
engine/trainer: task=detect, mode=train, model=yolov8m.pt, data=/content/drive/MyDrive/fiber_optic_box_inspection/dataset/data.yaml, epochs=100, time=None, patience=10, batch=25, imgsz=896, save=True, save_period=5, cache=True, device=0, workers=16, project=/content/drive/MyDrive/fiber_optic_box_inspection/runs_yolo, name=v8m_batch25_imgsz896_, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, sav

train: Scanning /content/drive/MyDrive/fiber_optic_box_inspection/dataset/data_label_v2_ALL... 7858 images, 501 backgrounds, 0 corrupt: 100%|██████████| 7858/7858 [00:27<00:00, 287.35it/s]

train: WARNING ⚠️ /content/drive/MyDrive/fiber_optic_box_inspection/dataset/data_label_v2_ALL/2022_11_03_15_29_32-HNIP456107HO.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /content/drive/MyDrive/fiber_optic_box_inspection/dataset/data_label_v2_ALL/2022_11_10_14_59_59-HNIP120595HO.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /content/drive/MyDrive/fiber_optic_box_inspection/dataset/data_label_v2_ALL/2022_11_11_09_10_09-PYNP016064HO.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /content/drive/MyDrive/fiber_optic_box_inspection/dataset/data_label_v2_ALL/2022_11_14_16_04_38-HNIP548102HO.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /content/drive/MyDrive/fiber_optic_box_inspection/dataset/data_label_v2_ALL/2022_11_16_08_43_48-HDGP141064HO.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /content/drive/MyDrive/fiber_optic_box_inspection/dataset/data_label_v2_ALL/2022_11_16_10_35_33-GLIP038152HO.jpg: 1 duplicate labels removed
train: WARNING ⚠️ /content/drive/MyDrive/fiber

train: New cache created: /content/drive/MyDrive/fiber_optic_box_inspection/dataset/data_label_v2_ALL.cache
WARNING ⚠️ cache='ram' may produce non-deterministic training results. Consider cache='disk' as a deterministic alternative if your disk space allows.


train: Caching images (12.4GB RAM): 100%|██████████| 7858/7858 [00:25<00:00, 309.89it/s]


...

                   all       1684      17744      0.631      0.553      0.539      0.358

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     76/100      19.2G     0.3264     0.2404     0.8792        201        896: 100%|██████████| 315/315 [05:38<00:00,  1.07s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 34/34 [00:23<00:00,  1.48it/s]


                   all       1684      17744      0.599      0.572       0.54       0.36

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     77/100      20.4G     0.3251     0.2368     0.8776        567        896:  48%|████▊     | 151/315 [02:43<02:57,  1.08s/it]


KeyboardInterrupt: 

## Resume Train

In [ ]:
model = YOLO('/content/drive/MyDrive/fiber_optic_box_inspection/runs_yolo/v8m_batch25_imgsz896_/weights/last.pt')
results = model.train(resume=True)

Ultralytics 8.3.111 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (NVIDIA L4, 22693MiB)
engine/trainer: task=detect, mode=train, model=/content/drive/MyDrive/fiber_optic_box_inspection/runs_yolo/v8m_batch25_imgsz896_/weights/last.pt, data=/content/drive/MyDrive/fiber_optic_box_inspection/dataset/data.yaml, epochs=100, time=None, patience=10, batch=25, imgsz=896, save=True, save_period=5, cache=True, device=0, workers=16, project=/content/drive/MyDrive/fiber_optic_box_inspection/runs_yolo, name=v8m_batch25_imgsz896_, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=/content/drive/MyDrive/fiber_optic_box_inspection/runs_yolo/v8m_batch25_imgsz896_/weights/last.pt, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, s

100%|██████████| 755k/755k [00:00<00:00, 126MB/s]



                   from  n    params  module                                       arguments                     
  0                  -1  1      1392  ultralytics.nn.modules.conv.Conv             [3, 48, 3, 2]                 
  1                  -1  1     41664  ultralytics.nn.modules.conv.Conv             [48, 96, 3, 2]                
  2                  -1  2    111360  ultralytics.nn.modules.block.C2f             [96, 96, 2, True]             
  3                  -1  1    166272  ultralytics.nn.modules.conv.Conv             [96, 192, 3, 2]               
  4                  -1  4    813312  ultralytics.nn.modules.block.C2f             [192, 192, 4, True]           
  5                  -1  1    664320  ultralytics.nn.modules.conv.Conv             [192, 384, 3, 2]              
  6                  -1  4   3248640  ultralytics.nn.modules.block.C2f             [384, 384, 4, True]           
  7                  -1  1   1991808  ultralytics.nn.modules.conv.Conv             [384

100%|██████████| 5.35M/5.35M [00:00<00:00, 327MB/s]


AMP: checks passed ✅
train: Fast image access ✅ (ping: 0.3±0.1 ms, read: 0.2±0.1 MB/s, size: 122.3 KB)


...

                   all       1684      17744      0.637      0.533      0.541      0.362

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


     97/100      19.3G     0.8101     0.4601      1.157         73        896: 100%|██████████| 315/315 [05:28<00:00,  1.04s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 34/34 [00:23<00:00,  1.47it/s]


                   all       1684      17744      0.589      0.562       0.54      0.362
EarlyStopping: Training stopped early as no improvement observed in last 10 epochs. Best results observed at epoch 87, best model saved as best.pt.
To update EarlyStopping(patience=10) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.

11 epochs completed in 1.094 hours.
Optimizer stripped from /content/drive/MyDrive/fiber_optic_box_inspection/runs_yolo/v8m_batch25_imgsz896_/weights/last.pt, 52.2MB
Optimizer stripped from /content/drive/MyDrive/fiber_optic_box_inspection/runs_yolo/v8m_batch25_imgsz896_/weights/best.pt, 52.2MB

Validating /content/drive/MyDrive/fiber_optic_box_inspection/runs_yolo/v8m_batch25_imgsz896_/weights/best.pt...
Ultralytics 8.3.111 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (NVIDIA L4, 22693MiB)
Model summary (fused): 92 layers, 25,939,927 parameters, 0 gradients, 79.2 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 34/34 [00:22<00:00,  1.48it/s]


                   all       1684      17744      0.596      0.563      0.546      0.364
                   1.0        136        136      0.725      0.904      0.855       0.73
                   1.1         61         61      0.516      0.734       0.68      0.533
                   1.2         35         35       0.41      0.543      0.544      0.356
                   1.7         20         21      0.342      0.524      0.376      0.246
                   2.0        260        262      0.922      0.954      0.968      0.868
                   2.3          1          1          0          0          0          0
                   3.0        248        248      0.933      0.956      0.985      0.648
                   3.1          2          2          1      0.955      0.995      0.423
                   5.0        249        490      0.918      0.984      0.977      0.691
                   5.1          6          6       0.45      0.833      0.644      0.211
                   6.

# what

In [ ]:
import torch
torch.cuda.empty_cache()